<a href="https://colab.research.google.com/github/carlibeisel/DCEW_waterquality/blob/main/DCEW_climatedata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installs geemap package
!pip install pycrs
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
try:
    import wxee
except ImportError:
    print('wxee package not installed. Installing...')
    subprocess.check_call(['python','-m','pip','install','wxee'])
  #!pip install eemont

import ee
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project = 'ee-tvirrigation')

from google.colab import drive
drive.mount('/content/gdrive')

import wxee
import os

  Preparing metadata (setup.py) ... done
  Created wheel for pycrs: filename=PyCRS-1.0.2-py3-none-any.whl size=32687 sha256=ecd33191f7e1d39ddc43d70854700568e2d0e0a4b89b79af9ef0e70feb373e25
  Stored in directory: /root/.cache/pip/wheels/47/1d/70/7a5bdf33347e7c75e95b06b1fa38f076a59a9506653cc24aff
Successfully built pycrs
wxee package not installed. Installing...


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


Mounted at /content/gdrive


In [2]:
!pip install geopandas
import geopandas as gpd
import json
import os
import numpy as np
import pandas as pd
!pip install pycrs
import pycrs
import seaborn as sns
import matplotlib.pyplot as plt

#Connect to Google Drive if you want to export images
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#DCEW shapefile

shp_file = '/content/drive/MyDrive/Classes/Field Hydrology/Hydro Field Course Working Files/Assignments/7. Data Analysis/Water Quality/Data/boundDCEW/boundDCEW.shp'
mason = emap.shp_to_ee(shp_file) # converts shapefile to feature in GEE

map = emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(ee.Image().paint(mason, 0, 2), {})
map.addLayerControl()
map

Map(center=[43.615, -116.2023], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [10]:
# ------------------------------------#
# Daily Precipitation in Mason Drainage
# ------------------------------------#

# units = mm

mason_precip = []
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').filterDate('2014-01-01', '2024-11-01')
gridmet_daily = gridmet.select('pr').map(lambda image: image.clip(mason))
mason_precip = gridmet_daily.toList(gridmet_daily.size())
mason_precip = ee.ImageCollection(mason_precip)
out_stats = os.path.join('/content/drive/MyDrive/Classes/Field Hydrology/Hydro Field Course Working Files/Assignments/7. Data Analysis/Water Quality/Data/dcew_gridmet_precip.csv')
emap.zonal_statistics(mason_precip, mason, out_stats, statistics_type='MEAN', scale=1000)

df = pd.read_csv('/content/drive/MyDrive/Classes/Field Hydrology/Hydro Field Course Working Files/Assignments/7. Data Analysis/Water Quality/Data/dcew_gridmet_precip.csv')
df = df.T.reset_index()
df.columns = ['Date', 'Precip (mm)']
df['Date'] = df['Date'].str.replace('_pr', '', regex=False)
df.to_csv('/content/drive/MyDrive/Classes/Field Hydrology/Hydro Field Course Working Files/Assignments/7. Data Analysis/Water Quality/Data/dcew_gridmet_precip.csv', index=False)

df = pd.read_csv('/content/drive/MyDrive/Classes/Field Hydrology/Hydro Field Course Working Files/Assignments/7. Data Analysis/Water Quality/Data/dcew_gridmet_precip.csv')

df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d', errors='coerce')
df['Precip (mm)'] = pd.to_numeric(df['Precip (mm)'], errors='coerce')
df['Precip (inches)'] = df['Precip (mm)'] * 0.0393701
df.to_csv('/content/drive/MyDrive/Classes/Field Hydrology/Hydro Field Course Working Files/Assignments/7. Data Analysis/Water Quality/Data/dcew_gridmet_precip.csv', index=False)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Classes/Field Hydrology/Hydro Field Course Working Files/Assignments/7. Data Analysis/Water Quality/Data/dcew_gridmet_precip.csv


In [11]:
# -------------------- #
# Export Monthly Precip
# -------------------- #

df = pd.read_csv('/content/drive/MyDrive/Classes/Field Hydrology/Hydro Field Course Working Files/Assignments/7. Data Analysis/Water Quality/Data/dcew_gridmet_precip.csv')
df['Date'] = pd.to_datetime(df['Date'])
df_monthly = df.resample('M', on='Date').sum(numeric_only=True)
df_monthly.index = df_monthly.index.strftime('%Y-%m')
df_monthly.to_csv('/content/drive/MyDrive/Classes/Field Hydrology/Hydro Field Course Working Files/Assignments/7. Data Analysis/Water Quality/Data/dcew_monthly_precip.csv')